# 04 — Hyperparameter Tuning & Model Saving
RandomizedSearchCV on 6 models, evaluate the best, and save the final pipeline.

In [1]:
import sys
sys.path.insert(0, '..')

import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)

from src.preprocessing import load_and_clean, build_preprocessor

## 1. Data Setup

In [2]:
df = load_and_clean()

X = df.drop('Risk', axis=1)
y = df['Risk'].map({'good': 1, 'bad': 0})

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'Train: {X_train.shape}  |  Test: {X_test.shape}')

Train: (417, 9)  |  Test: (105, 9)


## 2. Parameter Grids

In [4]:
log_reg_params = {
    'penalty': ['l1', 'l2'],
    'C'      : np.logspace(-3, 3, 30),
    'solver' : ['liblinear', 'saga']
}

svm_params = {
    'C'     : np.logspace(-2, 2, 30),
    'kernel': ['linear', 'rbf'],
    'gamma' : ['scale', 'auto'] + list(np.logspace(-3, 1, 10))
}

rf_params = {
    'n_estimators'   : np.arange(200, 800, 100),
    'max_depth'      : [None] + list(np.arange(4, 20, 2)),
    'min_samples_split': np.arange(2, 10),
    'min_samples_leaf' : np.arange(1, 6),
    'max_features'   : ['sqrt', 'log2']
}

gb_params = {
    'n_estimators'     : np.arange(150, 600, 50),
    'learning_rate'    : np.linspace(0.03, 0.3, 30),
    'max_depth'        : np.arange(2, 6),
    'min_samples_split': np.arange(2, 10),
    'min_samples_leaf' : np.arange(1, 6),
    'subsample'        : np.linspace(0.6, 1.0, 10)
}

xgb_params = {
    'n_estimators'   : np.arange(200, 800, 100),
    'learning_rate'  : np.linspace(0.03, 0.3, 30),
    'max_depth'      : np.arange(3, 10),
    'subsample'      : np.linspace(0.6, 1.0, 10),
    'colsample_bytree': np.linspace(0.6, 1.0, 10),
    'min_child_weight': np.arange(1, 6),
    'reg_alpha'      : np.linspace(0, 0.5, 10),
    'reg_lambda'     : np.linspace(0.5, 2, 10)
}

lgbm_params = {
    'n_estimators'    : [200, 400],
    'learning_rate'   : [0.05, 0.1],
    'num_leaves'      : [31, 63],
    'max_depth'       : [-1, 8],
    'min_child_samples': [20, 40],
    'subsample'       : [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

## 3. RandomizedSearchCV — optimise Recall

In [5]:
preprocessor = build_preprocessor()

randomcv_models = [
    ('LogReg', LogisticRegression,         log_reg_params),
    ('SVM',    SVC,                        svm_params),
    ('RF',     RandomForestClassifier,     rf_params),
    ('GB',     GradientBoostingClassifier, gb_params),
    ('XGB',    XGBClassifier,              xgb_params),
    ('LGBM',   LGBMClassifier,             lgbm_params),
]

best_models  = {}
tune_results = {}

for name, ModelClass, params in randomcv_models:
    pipe = Pipeline([('prep', preprocessor), ('model', ModelClass())])

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions={f'model__{k}': v for k, v in params.items()},
        n_iter=50,
        scoring='recall',
        cv=5,
        n_jobs=-1,
        random_state=42
    )
    search.fit(X_train, y_train)

    best_models[name] = search.best_estimator_
    y_pred = search.predict(X_test)

    tune_results[name] = {
        'Recall'   : recall_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'F1'       : f1_score(y_test, y_pred),
        'Accuracy' : accuracy_score(y_test, y_pred)
    }
    print(f'{name} done')

LogReg done
SVM done
RF done
GB done
XGB done


KeyboardInterrupt: 

In [6]:
print('Tuning Results (sorted by Recall):')
pd.DataFrame(tune_results).T.sort_values('Recall', ascending=False)

Tuning Results (sorted by Recall):


,Recall,Precision,F1,Accuracy
LogReg,1.000000,0.561905,0.719512,0.561905
SVM,1.000000,0.561905,0.719512,0.561905
RF,0.881356,0.666667,0.759124,0.685714
XGB,0.813559,0.750000,0.780488,0.742857
GB,0.762712,0.737705,0.750000,0.714286


In [7]:
for model_name, pipe in best_models.items():
    print(f'\n--- Best params for {model_name} ---')
    print(pipe.named_steps['model'].get_params())


--- Best params for LogReg ---
{'C': np.float64(0.002592943797404667), 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': 0.0, 'max_iter': 100, 'n_jobs': None, 'penalty': 'l1', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

--- Best params for SVM ---
{'C': np.float64(0.018873918221350976), 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': np.float64(0.46415888336127775), 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}

--- Best params for RF ---
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': np.int64(4), 'max_features': 'log2', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': np.int64(4), 'min_samples_split': np.int64(6), 'min_weight_fraction_lea

## 4. Final Model — Best XGBoost with Tuned Hyperparameters

In [9]:
final_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    colsample_bytree=0.6888888888888889,
    learning_rate=0.03,
    max_depth=4,
    min_child_weight=4,
    n_estimators=200,
    reg_alpha=0.3333333333333333,
    reg_lambda=1.0,
    subsample=0.9111111111111111
)

final_pipe = Pipeline([
    ('prep',  build_preprocessor()),
    ('model', final_model)
])
final_pipe.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('prep', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('nom', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers conta

## 5. Final Evaluation

In [10]:
y_te_pred = final_pipe.predict(X_test)
y_te_prob = final_pipe.predict_proba(X_test)[:, 1]

print('Final XGBoost — Test Set Performance')
print(f'  Accuracy : {accuracy_score(y_test, y_te_pred):.4f}')
print(f'  Precision: {precision_score(y_test, y_te_pred):.4f}')
print(f'  Recall   : {recall_score(y_test, y_te_pred):.4f}')
print(f'  F1 Score : {f1_score(y_test, y_te_pred):.4f}')
print(f'  ROC-AUC  : {roc_auc_score(y_test, y_te_prob):.4f}')

Final XGBoost — Test Set Performance
  Accuracy : 0.7429
  Precision: 0.7500
  Recall   : 0.8136
  F1 Score : 0.7805
  ROC-AUC  : 0.7594


## 6. Save Model

In [ ]:
os.makedirs('saved_models', exist_ok=True)
joblib.dump(final_pipe, 'saved_models/Xgboost_model1.joblib')
print('Model saved to saved_models/Xgboost_model1.joblib')

## 7. Sanity Check — Load & Predict

In [3]:
pipe_loaded = joblib.load('saved_models/Xgboost_model1.joblib')

new_data = pd.DataFrame([{
    'Age': 30, 'Credit amount': 2200, 'Duration': 7,
    'Sex': 'male', 'Housing': 'own',
    'Saving accounts': 'rich', 'Checking account': 'rich',
    'Purpose': 'business', 'Job': '3'
}])

prob_good = pipe_loaded.predict_proba(new_data)[0, 1]
print(f'Default risk: {1 - prob_good:.2%}')

Default risk: 6.18%
